# Übung 7 - Arnoldi-Verfahren

In [1]:
import numpy as np

### Prozedur zur Berechnung von Skalarprodukten

In [2]:
def skp(u,v):
    n = len(u)
    res = 0
    for j in range(n):
        res = res + u[j]*v[j]
        
    return res

### Arnoldi-Verfahren

**Hinweise:**
- Wir speichern den Vektor \\(\tilde{v}^{m+1}\\), der sukzessive zum nächsten Vektor der ONB verändert wird, jeweils in dem numpy-array `v_new` und den Wert \\(h_{m+1,m} = \lVert \tilde{v}^{m+1} \rVert\\) in der Variable `h_new`.
- Wir führen das Arnoldi-Verfahren bis zum Erreichen des Invarianzindex durch. Dementsprechend wissen wir nicht vorab, wie viele Schleifendurchläufe wir berechnen müssen, verwenden also eine `while`-Schleife. Wir erkennen das Erreichen des Invarianzindex daran, dass \\(h_{m+1,m}=0\\) gilt und nutzen dies als Abbruchkriterium. Da wir aufgrund von Rundungsfehlern nie exakt den Wert Null für \\(h_{m+1,m}\\) erhalten, überprüfen wir stattdessen, ob \\(\lvert h_{m+1,m} \rvert < 10^{-10} \\) gilt.
- Wir speichern die Matrizen \\(V_m\\) und \\(H_m\\), \\(m=1,2,...\\) in jeweils einem numpy-array `V` bzw. `H`, das wir sukzessive vergrößern. Dazu eignet sich der `pad` Befehl. Für eine Matirx `S` ergänzt der Befehl `np.pad(S, ((z1,z2),(s1,s2)) )` die Matrix `S` um `z1` Zeilen vor der ersten Zeile, `z2` Zeilen nach der letzten Zeile, `s1` Spalten vor der ersten Spalte und `s1` Spalten nach der letzten Spalte. 
- Damit der `pad` Befehl von Anfang an funktioniert, müssen wir auch \\(H_1\\) und \\(V_1\\) explizit als \\( (1\times1) \\)- bzw. \\( (n\times1) \\)-**Matrix** initialisieren.

In [36]:
def arnoldi(A,b):
    beta = np.sqrt(skp(b,b))
    # Erstellung von V_1
    V = np.zeros([len(b),1])
    V[:,0] = b/beta
    
    # Ersten Schleifendurchgang bis zur Berechnung von h_new = Norm von v_new durchmachen:
    m = 1
    # Erster zu orthogonalisierender Vektor:
    v_new = A@V[:,0]
    # H als (1 x 1)-Matrix initialsieren, Vektor v_new orthogonalisieren
    H = np.zeros([1,1])
    H[0,0] = skp(V[:,0],v_new) 
    v_new = v_new - H[0,0] * V[:,0]
    # h_new = Norm des Vektors v_new
    h_new = np.sqrt( skp(v_new,v_new) )
    
    while abs(h_new) > 1e-10:
        # Aktueller Stand: 
        # - In V und H stecken die schon berechneten Matrizen V_m (n x m) und H_m (m x m).
        # - v_new enthält den (noch nicht normierten) Basisvektor v_{m+1}
        # - h_new enthält die Norm von v_new. h_new ist nicht Null 
        #   --> kein Abbruch, normiertere Variante von v_new ist tatsächlich ein neuer Basisvektor
        # ToDo in einem Schleifendurchlauf:
        # 1.) Ergänze V = V_m um die zusätzliche Spalte v_new/h_new --> V wird zu V_{m+1} (n x m+1)
        # 2.) Setze m = m + 1 --> In V und H stecken jetzt V_m (n x m) und H_{m-1} (m-1 x m-1)
        # 3.) Bereite H_{m} (m x m) vor: 
        #     a) Ergänze H = H_{m-1} um zusätzliche Spalte und Zeile --> H wird zu H_m (m x m)
        #     b) Trage den Wert h_new in die letzte Zeile, vorletzte Spalte ein 
        # 4.) Berechne neuen Vektor im Krylov-Raum A v_m, orthogonalisiere ihn --> v_new = Kandidat für v_{m+1}.
        #     Trage dabei die Skalaprodukte in die letzte Spalte von H_m ein 
        # 5.) Berchne h_new = Norm von v_new
        
        # Schritt 1: neuer Vektor der ONB
        v_new = v_new/h_new
        V = np.pad(V, ((0,0),(0,1)) )
        V[:,m] = v_new
        # Schritt 2: m erhöhen
        m = m + 1
        # Schritt 3: H_m vorbereiten
        H = np.pad(H, ((0,1),(0,1)) )
        H[m-1,m-2] = h_new
        # Schritt 4: Neuen Basis-Vektor berechnen, orthogonalisieren, H_m befüllen
        v_new = A@V[:,m-1]
        for i in range(m):
            H[i,m-1] = skp( V[:,i] , v_new)
            v_new = v_new - H[i,m-1] * V[:,i]
        # Schritt 5: Norm von v_new
        h_new = np.sqrt( skp(v_new,v_new) )
    
    return V,H
    

Probe: Wir verwenden zur Überprüfung die Matrix \\(A\\) und den Vektor \\(b\\) aus Beispiel 3.44 im Skript:

In [45]:
A = np.array([ [2,-1,0],[1,1,1],[3,0,-1] ])
b = np.array( [0,6,0] )

In [46]:
V,H = arnoldi(A,b)
print('V:')
print(V)
print('H:')
print(H)

V:
[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
H:
[[ 1. -1. -1.]
 [ 1.  2.  0.]
 [ 0.  3. -1.]]


Wir können auch 'von Hand' überprüfen, ob alle Anforderungen erfüllt sind, d.h.:
- Gilt die Arnoldi-Relation (in der Variante für den erreichten Invarianzindex) \\(AV_m = V_mH_m\\)?
- Hat `V` orthogonale Spalten?
- Hat `H` Hessenberg-Gestalt? --> Direkt erkennbar.

In [18]:
print('AV-VH = ')
print(A@V-V@H)
print('V^TV - I_m = ')
print(V.T@V - np.eye(V.shape[1]))

AV-VH = 
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
V^TV - I_m = 
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


Zu Demonstrationszwecken: Wir probieren unser Arnoldi-Verfahren mit einem Eigenvektor von \\(A\\) als Vektor \\(b\\) aus: 

In [56]:
b = np.array([-0.11341694337636568,-0.3788410392636101,1])
V,H = arnoldi(A,b)
print('V:')
print(V)
print('H:')
print(H)

V:
[[-0.10546951]
 [-0.35229461]
 [ 0.92992725]]
H:
[[-1.34025083]]


Wie zu erwarten bricht unser Verfahren nach einem Schritt ab, da der Invarianzindex \\(\ell\\) von \\(\mathcal{K}_m(A,b)\\) gleich \\(1\\) ist falls \\(b\\) ein Eigenvektor von \\(A\\) ist.

### Erweiterung zum FOM-Verfahren

Für das FOM-Verfahren müssen wir die obige Prozedur lediglich um die Berechnung der aktuellen Iterierten \\(x^{(m)}\\) aus `V` und `H` ergänzen.

Zur Erinnerung die nötigen Schritte im FOM-Verfahren:
- Löse LGS \\(H_m y^{(m)} = \beta e^1\\) (\\(e^1\\) erster Einheitsvektor in \\(\mathbb{R}^m\\))
- Setze \\(x^{(m)} = V_m y^{(m)}\\).
Zur Lösung des LGS nutzen wir einfachheitshalber die numpy-Routine `np.linalg.solve`.

In [11]:
def FOM(A,b):
    beta = np.sqrt(skp(b,b))
    # Erstellung von V_1
    V = np.zeros([len(b),1])
    V[:,0] = b/beta
    
    # Ersten Schleifendurchgang bis zur Berechnung von h_new = Norm von v_new durchmachen:
    m = 1
    # Erster zu orthogonalisierender Vektor:
    v_new = A@V[:,0]
    # H als (1 x 1)-Matrix initialsieren, Vektor v_new orthogonalisieren
    H = np.zeros([1,1])
    H[0,0] = skp(V[:,0],v_new) 
    v_new = v_new - H[0,0] * V[:,0]
    # h_new = Norm des Vektors v_new
    h_new = np.sqrt( skp(v_new,v_new) )
    
    m=1
    
    # Erste Approximation berechnen
    # Ersten Einheitsvektor in R^m erstellen:
    vec = np.zeros(m)
    vec[0] = 1
    # LGS Hy = beta*e^1 lösen
    y = np.linalg.solve(H,beta*vec)
    # x = Vy bestimmen
    x = V@y
    # Aktuelle Iterierte ausgeben
    print('m =',m,'--> Approximation',x)
    
    while abs(h_new) > 1e-10:
        # Aktueller Stand: 
        # - In V und H stecken die schon berechneten Matrizen V_m (n x m) und H_m (m x m).
        # - v_new enthält den (noch nicht normierten) Basisvektor v_{m+1}
        # - h_new enthält die Norm von v_new. h_new ist nicht Null 
        #   --> kein Abbruch, normiertere Variante von v_new ist tatsächlich ein neuer Basisvektor
        # ToDo in einem Schleifendurchlauf:
        # 1.) Ergänze V = V_m um die zusätzliche Spalte v_new/h_new --> V wird zu V_{m+1}
        # 2.) Setze m = m + 1 --> In V und H stecken jetzt V_m (n x m) und H_{m-1} (m-1 x m-1)
        # 3.) Bereite H_{m} (m x m) vor: 
        #     a) Ergänze H = H_{m-1} um zusätzliche Spalte und Zeile --> H wird zu H_m (m x m)
        #     b) Trage den Wert h_new in die letzte Zeile, vorletzte Spalte ein 
        # 4.) Berechne neuen Vektor im Krylov-Raum A v_m, orthogonalisiere ihn --> v_new = Kandidat für v_{m+1}.
        #     Trage dabei die Skalaprodukte in die letzte Spalte von H_m ein 
        # 5.) Berchne h_new = Norm von v_new
        # 6.) FÜr FOM: Berechne neue Iterierte
        
        # Schritt 1: neuer Vektor der ONB
        v_new = v_new/h_new
        V = np.pad(V, ((0,0),(0,1)) )
        V[:,m] = v_new
        # Schritt 2: m erhöhen
        m = m + 1
        # Schritt 3: H_m vorbereiten
        H = np.pad(H, ((0,1),(0,1)) )
        H[m-1,m-2] = h_new
        # Schritt 4: Neuen Basis-Vektor berechnen, orthogonalisieren, H_m befüllen
        v_new = A@V[:,m-1]
        for i in range(m):
            H[i,m-1] = skp( V[:,i] , v_new)
            v_new = v_new - H[i,m-1] * V[:,i]
        # Schritt 5: Norm von v_new
        h_new = np.sqrt( skp(v_new,v_new) )
        
        # Schritt 6
        # Ersten Einheitsvektor in R^m erstellen:
        vec = np.zeros(m)
        vec[0] = 1
        # LGS Hy = beta*e^1 lösen
        y = np.linalg.solve(H,beta*vec)
        # x = Vy bestimmen
        x = V@y
        # Aktuelle Iterierte ausgeben
        print('m =',m,'--> Approximation',x)
        
    return x
    

Probe: Erneut verwenden wir zur Überprüfung die Matrix \\(A\\) und den Vektor \\(b\\) aus Beispiel 3.44 im Skript:

In [19]:
A = np.array([ [2,-1,0],[1,1,1],[3,0,-1] ])
b = np.array( [0,6,0] )
x = FOM(A,b)

m = 1 --> Approximation [0. 6. 0.]
m = 2 --> Approximation [2. 4. 0.]
m = 3 --> Approximation [1. 2. 3.]


Zusätzlich können wir überprüfen, ob die letzte Approximation tatsächlich eine Lösung des LGS \\(Ax=b\\) ist:

In [20]:
print('Probe: b-Ax = ',b-A@x)

Probe: b-Ax =  [0. 0. 0.]
